# 构建一条链

In [3]:
openai_api_key="你的api-key"
base_url="填入你的转接"

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=openai_api_key,
                 
    base_url=base_url,
                temperature=0.0)


In [5]:
from langchain_core.prompts import ChatPromptTemplate
template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"
text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
# 构建提示模板消息列表
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

# format_message传参
messages  = chat_prompt.format_messages(input_language="中文", output_language="英文", text=text)

In [6]:
# 输出解析
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()


In [7]:
# 链
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language":"中文", "output_language":"英文","text": text})

"I carried luggage heavier than my body and waded into the depths of the Nile. After a few flashes of lightning, I saw a cluster of halos and wasn't sure if this was the place."

# 构建检索问答

In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
embedding = OpenAIEmbeddings(api_key="sk-vBHSvmTQFTvt7dLessW5JGQGOv3Oo88lPawcZIBQnPUHSEuD",
                    base_url="https://api.chatanywhere.tech/v1",
                    model="text-embedding-3-small"
                    ) 
persist_directory = 'data_base/vector_db/chroma'

In [9]:
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)
print(f"向量库中存储的数量：{vectordb._collection.count()}")


向量库中存储的数量：20


In [10]:
question = "什么是机器学习?"
docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(docs)}")


检索到的内容数：3


In [11]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")


检索到的第0个内容: 
 前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解,所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以......本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”。
使用说明
南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；对于初学机器学习的小白，西瓜书第1章和第2章的公式强烈不建议深究，简单过一下即可，等你学得
-----------------------------------------------------
检索到的第1个内容: 
 →_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解第2 版》
←_←
目录
第1 章绪论
1
1.1
引言. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
1
1.2
基本术语
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
1
1.3
假设空间
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
3
1.4
归纳偏好
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [19]:
from langchain.prompts import PromptTemplate

template = """只使用以下文本进行回答。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})
question_1 = "什么是西瓜书？"
question_2 = "王阳明是谁？"

In [15]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])
llm.invoke(question_1).content

大模型+知识库后回答 question_1 的结果：
西瓜书是机器学习领域的经典入门教材，由周志华老师编写。它提供了机器学习知识的基本理论和方法。
谢谢你的提问！


'西瓜书通常指的是《西瓜书：现代操作系统》（Operating Systems: Internals and Design Principles）这本计算机科学领域的经典教材，作者是Abraham Silberschatz、Peter Baer Galvin和Greg Gagne。这本书涵盖了操作系统的内部原理和设计原则，对于计算机科学和工程专业的学生来说是一本非常重要的参考书。"西瓜书"因为封面上有一个西瓜瓤的图案而得名，成为学习操作系统课程时的常用教材。'

In [20]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])
llm.invoke(question_2).content

大模型+知识库后回答 question_2 的结果：
我不知道，谢谢你的提问！


'王阳明（1472-1529）是明朝时期的思想家、政治家和武将。他提出了“知行合一”的理论，强调心性的重要性，主张通过内心的觉悟来实现个人修养和社会改良。他在中国思想史上影响深远，被视为儒家思想的代表人物之一。'

# 对话检索

In [18]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)


retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "我可以学习到关于机器学习的知识吗？"
result = qa({"question": question})
print(result['answer'])


可以的，你可以通过学习《机器学习公式详解第2 版》获得关于机器学习的知识。这本书是以《机器学习》（西瓜书）为基础进行展开的，特别是对一些难懂的公式进行了解析和补充推导细节。所以，如果你想深入了解和学习机器学习，这本书可能会对你有所帮助。
